In [1]:
# Load packages bigsnpr and bigstatsr
library(bigsnpr)
library(tidyverse)
library(data.table)

# Load bigSNP objects in R session
cat('\nLoading bigSNP objects...')
for (chr in 1:22) {
    cat(paste('\nCurrently working on chromosome',chr))
    obj.bigSNP <- snp_attach(paste0("/mnt/stsi/stsi1/ptseng/UKBB_Resources/ldpred2/chr",chr,"/temp.rds"))
    #str(obj.bigSNP, max.level = 2, strict.width = "cut")
  
    if (chr == 1) {
        G   <- c(obj.bigSNP$genotypes)
        map <- obj.bigSNP$map
    } else {
        G   <- append(G,obj.bigSNP$genotypes)
        map <- bind_rows(map,obj.bigSNP$map)
    }
}

# Fixing column values
map$chromosome <- as.integer(map$chromosome)
map$physical.pos <- as.integer(map$physical.pos)

cat('\nDONE\n')

# Get aliases for useful slots
CHR <- map$chromosome
ind.N <- nrow(G[[1]])
NCORES <- nb_cores()

# Set individuals
ind.val <- sample(ind.N, ind.N*4/5)
ind.test <- setdiff(ind.N, ind.val)

# Read external summary statistics
cat('\nLoading summary statistics...')
sumstats <- bigreadr::fread2("../PD_ukb_step2_BT_ltfh.regenie.corrected.reduced")
cat('\nDONE\n')
# Matching Sumstats with Map
cat('\nMatching summary statistics with map data...')
sumstats <- sumstats[,c('CHROM', 'GENPOS', 'ALLELE0', 'ALLELE1', 'A1FREQ', 'BETA', 'SE')]
names(sumstats) <- c("chr", "pos", "a0", "a1", "a1_freq", "beta", "beta_se")
sumstats['n_eff'] <- 419488

map <- map[,c('chromosome','physical.pos','allele1','allele2')]
names(map) <- c("chr", "pos", "a0", "a1")

info_snp <- snp_match(sumstats, map)
info_snp <- tidyr::drop_na(tibble::as_tibble(info_snp))
cat('\nDONE\n')

# Set Temporary File
tmp <- tempfile(tmpdir = "tmp-data")

Loading required package: bigstatsr

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose





Loading bigSNP objects...
Currently working on chromosome 1
Currently working on chromosome 2
Currently working on chromosome 3
Currently working on chromosome 4
Currently working on chromosome 5
Currently working on chromosome 6
Currently working on chromosome 7
Currently working on chromosome 8
Currently working on chromosome 9
Currently working on chromosome 10
Currently working on chromosome 11
Currently working on chromosome 12
Currently working on chromosome 13
Currently working on chromosome 14
Currently working on chromosome 15
Currently working on chromosome 16
Currently working on chromosome 17
Currently working on chromosome 18
Currently working on chromosome 19
Currently working on chromosome 20
Currently working on chromosome 21
Currently working on chromosome 22
DONE

Loading summary statistics...
DONE

Matching summary statistics with map data...

1,014,228 variants to be matched.

146,667 ambiguous SNPs have been removed.

Some duplicates were removed.

867,179 variants have been matched; 0 were flipped and 0 were reversed.




DONE


In [2]:
sumstats <- info_snp

In [3]:
head(sumstats) 

chr,pos,a0,a1,a1_freq,beta,beta_se,n_eff,_NUM_ID_.ss,_NUM_ID_
<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,79807,G,A,0.000051235,0.055852397,0.02057280,419488,1,1
1,129672,A,G,0.000118492,0.040025593,0.01477910,419488,2,2
1,601158,GAGTA,G,0.000854688,0.013466354,0.00462319,419488,3,3
1,720026,C,T,0.000020316,0.058603689,0.02729160,419488,4,4
1,720663,T,C,0.000357367,0.019138239,0.00895003,419488,5,5
1,742594,C,T,0.001727370,-0.006687322,0.00368591,419488,6,6


In [4]:
# Load Correlation Info from File
cat('\nLoading correlation info...')
for (chr in 1:22) {

  # print(chr)

  ## indices in 'df_beta'
  ind.chr1 <- which(sumstats$chr == chr)
  ## indices in 'map_ldref'
  ind.chr2 <- sumstats$`_NUM_ID_`[ind.chr1]
  ## indices in 'corr_chr'
  ind.chr3 <- match(ind.chr2, which(map$chr == chr))
    
  cat(paste('\nreadRDS for chromosome',chr))
  corr0 <- readRDS(paste0("/mnt/stsi/stsi1/ptseng/UKBB_Resources/ldpred2/corr/chr", chr, ".rds"))[ind.chr3, ind.chr3]

  if (chr == 1) {
    df_beta <- sumstats[ind.chr1, c("beta", "beta_se", "n_eff", "_NUM_ID_")]
    ld <- Matrix::colSums(corr0^2)
    corr <- as_SFBM(corr0, tmp)
  } else {
    df_beta <- rbind(df_beta, sumstats[ind.chr1, c("beta", "beta_se", "n_eff", "_NUM_ID_")])
    ld <- c(ld, Matrix::colSums(corr0^2))
    corr$add_columns(corr0, nrow(corr))
  }
}
cat('\nDONE\n')


Loading correlation info...
readRDS for chromosome 1
readRDS for chromosome 2
readRDS for chromosome 3
readRDS for chromosome 4
readRDS for chromosome 5
readRDS for chromosome 6
readRDS for chromosome 7
readRDS for chromosome 8
readRDS for chromosome 9
readRDS for chromosome 10
readRDS for chromosome 11
readRDS for chromosome 12
readRDS for chromosome 13
readRDS for chromosome 14
readRDS for chromosome 15
readRDS for chromosome 16
readRDS for chromosome 17
readRDS for chromosome 18
readRDS for chromosome 19
readRDS for chromosome 20
readRDS for chromosome 21
readRDS for chromosome 22
DONE


In [6]:
(ldsc <- with(df_beta, snp_ldsc(ld, length(ld), chi2 = (beta / beta_se)^2,
                                  sample_size = n_eff, blocks = NULL)))
h2_est <- ldsc[["h2"]]

ERROR: Error in if (max(abs(pred - pred0)) < 1e-06) break: missing value where TRUE/FALSE needed


In [ ]:
# ldpred2 auto
multi_auto <- snp_ldpred2_auto(corr, df_beta, h2_init = h2_est,
                               vec_p_init = seq_log(1e-4, 0.9, 30),
                               ncores = NCORES)
beta_auto <- sapply(multi_auto, function(auto) auto$beta_est)

In [ ]:
library(data.table)

In [ ]:
fwrite(beta_auto,'ldpred2_output',sep=' ',row.names=FALSE)

In [ ]:
fwrite(df_beta,'ldpred2_statsfile',sep=' ',row.names=FALSE)